# 10 - Agentic RAG (Autonomous Agents)

**Complexity:** ⭐⭐⭐⭐⭐

**Use Cases:** Multi-step reasoning, BI dashboards, complex decision-making

**Key Features:**
- ReAct agent (Reasoning + Acting)
- Multiple tools (retriever, calculator, web search)
- Autonomous planning and execution
- Multi-step reasoning

**Example:**
```
Query: "If I have 10K docs and process 1M tokens/day, should I use OpenAI or HF?"

Agent:
1. Thought: Need cost calculation
   Action: Calculator → Estimate costs
2. Thought: Need comparison info
   Action: Knowledge Base → Retrieve comparison
3. Thought: Can now recommend
   Final Answer: "HuggingFace is better because..."
```

In [7]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import REACT_AGENT_PROMPT

print_section_header("Setup: Agentic RAG")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")


SETUP: AGENTIC RAG

✓ Loaded vector store from /Users/gianlucamazza/Workspace/notebooks/llm_rag/notebooks/advanced_architectures/../../data/vector_stores/openai_embeddings
✅ Setup complete!


## 2. Create Tools for Agent

In [8]:
from langchain_core.tools import tool
import numexpr

print_section_header("Agent Tools")

# Tool 1: Knowledge Base Retriever
@tool
def knowledge_base(query: str) -> str:
    """Search LangChain RAG knowledge base.
    Use for questions about RAG, embeddings, retrievers, LangChain."""
    docs = retriever.invoke(query)
    return format_docs(docs[:3])

# Tool 2: Calculator
@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions safely.
    Input should be a valid math expression like '2 + 2' or '100000 * 0.00002'.
    Examples: '10 + 5', '(100 * 2) / 3', '2 ** 8'"""
    try:
        # Use numexpr for safe evaluation
        result = numexpr.evaluate(expression).item()
        return str(result)
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"

# Tool 3: Web Search (if available)
tools = [knowledge_base, calculator]

try:
    from langchain_community.tools.tavily_search import TavilySearchResults
    
    @tool
    def web_search(query: str) -> str:
        """Search the web for current information.
        Use for queries outside the knowledge base."""
        search = TavilySearchResults(max_results=2)
        return search.invoke(query)
    
    tools.append(web_search)
    print("✓ Created 3 tools (retriever, calculator, web)")
except ImportError:
    print("✓ Created 2 tools (retriever, calculator)")

print("  Tools:")
for t in tools:
    print(f"    - {t.name}: {t.description[:60]}...")


AGENT TOOLS

✓ Created 3 tools (retriever, calculator, web)
  Tools:
    - knowledge_base: Search LangChain RAG knowledge base.
Use for questions about...
    - calculator: Calculate mathematical expressions safely.
Input should be a...
    - web_search: Search the web for current information.
Use for queries outs...


## 3. Create ReAct Agent

In [9]:
from langchain.agents import create_agent

print_section_header("Modern ReAct Agent (LangChain 1.0)")

# Create modern agent (built on LangGraph)
agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt="""You are a helpful AI assistant with expertise in RAG systems and embeddings.

You have access to these tools:
- knowledge_base: Search documentation about RAG, embeddings, and LangChain
- calculator: Perform mathematical calculations
- web_search: Search the web for current information (if available)

Guidelines:
1. Think step by step about what information you need
2. Use the appropriate tool(s) to gather information
3. Synthesize the results into a clear, concise answer
4. Always cite your sources when using the knowledge base

Be helpful and accurate!"""
)

print("✓ Modern ReAct agent created")
print("  - Built on LangGraph internally")
print("  - Automatic state management")
print("  - Message-based interface")
print("  - Streaming support")


MODERN REACT AGENT (LANGCHAIN 1.0)

✓ Modern ReAct agent created
  - Built on LangGraph internally
  - Automatic state management
  - Message-based interface
  - Streaming support


## 4. Test Agent - Simple Query

In [10]:
print_section_header("Test 1: Simple Query")

query = "What is a vector store?"
print(f"Query: '{query}'\n")
print("=" * 80)

# Modern message-based invocation
response = agent_executor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print("\n" + "=" * 80)

# Extract final answer from messages
final_answer = response["messages"][-1].content
print(f"\nFinal Answer: {final_answer}")


TEST 1: SIMPLE QUERY

Query: 'What is a vector store?'



Final Answer: A vector store is a specialized database designed to store and index vector representations of data, typically generated through embeddings. In the context of machine learning and natural language processing, embeddings are numerical representations of text or other data types that capture semantic meaning.

Here's how a vector store typically fits into the workflow:

1. **Load**: Data is loaded using document loaders.
2. **Split**: The data is divided into smaller chunks using text splitters, which makes it easier to manage and search.
3. **Store**: The smaller chunks are then stored in a vector store, where they can be indexed and searched efficiently.

Vector stores are essential for applications like information retrieval, where you need to quickly find relevant data based on similarity in the vector space. They enable efficient querying and retrieval of information based on the semantic content of the data ra

## 5. Test Agent - Multi-Step Reasoning

In [11]:
print_section_header("Test 2: Multi-Step")

query = """If embedding 100,000 documents costs $2 with OpenAI,
what's the cost per document? Then tell me if HuggingFace is cheaper."""

print(f"Query: {query}\n")
print("=" * 80)

# Modern message-based invocation
response = agent_executor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print("\n" + "=" * 80)

# Extract final answer
final_answer = response["messages"][-1].content
print(f"\nFinal Answer: {final_answer}")

print("\n💡 Agent used multiple tools to solve this!")


TEST 2: MULTI-STEP

Query: If embedding 100,000 documents costs $2 with OpenAI,
what's the cost per document? Then tell me if HuggingFace is cheaper.



Final Answer: The cost per document when embedding 100,000 documents with OpenAI is calculated as follows:

\[
\text{Cost per document} = \frac{\text{Total cost}}{\text{Number of documents}} = \frac{2}{100000} = 0.00002 \text{ USD} \text{ or } 0.02 \text{ cents per document.}
\]

Regarding Hugging Face, the pricing information indicates that they charge for inference endpoints starting from approximately $0.033 per CPU-hour. However, the exact cost per document can vary significantly based on the model used and the size of the documents being processed. 

In general, OpenAI's cost of $0.02 per document is lower than the starting cost of Hugging Face's inference endpoints, but the final comparison would depend on the specific usage and model requirements for Hugging Face.

For more detailed pricing, you can refer to the sources:
- [Emb

## 6. Agent with Memory

In [12]:
print_section_header("Conversational Agent")

print("✓ Using message-based conversation state\n")
print("  Modern LangChain 1.0 approach:")
print("  - No separate memory object needed")
print("  - State managed through message history")
print("  - Agent automatically maintains context\n")

# Maintain conversation history as a list of messages
conversation = []

# Multi-turn conversation
# Turn 1
query1 = "What are the embedding dimensions?"
print(f"User: {query1}\n")

conversation.append({"role": "user", "content": query1})
response1 = agent_executor.invoke({"messages": conversation})
conversation = response1["messages"]  # Update with full conversation

answer1 = conversation[-1].content
print(f"Agent: {answer1[:150]}...\n")

# Turn 2 (agent has context from turn 1)
query2 = "Which one is cheaper?"
print(f"\nUser: {query2}\n")

conversation.append({"role": "user", "content": query2})
response2 = agent_executor.invoke({"messages": conversation})
conversation = response2["messages"]  # Update with full conversation

answer2 = conversation[-1].content
print(f"Agent: {answer2[:150]}...")

print("\n✅ Agent maintained conversation context!")
print(f"   (Total messages in conversation: {len(conversation)})")


CONVERSATIONAL AGENT

✓ Using message-based conversation state

  Modern LangChain 1.0 approach:
  - No separate memory object needed
  - State managed through message history
  - Agent automatically maintains context

User: What are the embedding dimensions?

Agent: Embedding dimensions refer to the size of the vector representation used to encode data, such as text, images, or other types of information. In the c...


User: Which one is cheaper?

Agent: When considering the cost of embedding dimensions, "cheaper" can refer to several factors, including computational cost, memory usage, and training ti...

✅ Agent maintained conversation context!
   (Total messages in conversation: 8)


## Summary

**Flow:**
```
Query → Agent Reasoning Loop:
  1. Think about what to do
  2. Select tool to use
  3. Execute tool
  4. Observe result
  5. Repeat until answer found
→ Final Answer
```

**Advantages:**
✅ Autonomous decision-making  
✅ Multi-step reasoning  
✅ Tool composition  
✅ Handles complex queries  
✅ Extensible (add more tools)  

**Limitations:**
- Very slow (many LLM calls)
- Expensive (reasoning tokens)
- Can get stuck in loops
- Unpredictable behavior
- Hard to debug

**When to Use:**
- Complex multi-step tasks
- Requires multiple data sources
- Research/analysis workflows
- BI dashboards

**Production Tips:**
- Set max_iterations carefully
- Monitor token usage
- Add timeout mechanisms
- Implement fallbacks
- Use LangSmith for tracing
- Test edge cases thoroughly

**Next:** [11_comparison.ipynb](11_comparison.ipynb) - Benchmark all architectures